In [27]:
import pandas as pd
import numpy as np
import datetime as dt

In [28]:
etf_list = pd.read_sql_query("select distinct symbol from mkt", con = engine)

In [39]:
def update_return_sharpe(df, date, min_date, max_date):
    if date <= min_date:
        return None, None
    else:
        filter_data = df[(df['date'] >= date) & (df['date'] <= pd.to_datetime(max_date))].sort_values(by = 'date')
        open_close = filter_data.iloc[[0, -1], :]['close'].values
        cum_return = (open_close[1] - open_close[0])/open_close[0]
        hold_days = (date - min_date).days
        annual_return = (1 + cum_return)**(250/len(filter_data)) - 1
        filter_data['close_t-1'] = filter_data.shift(1)['close']
        filter_data['return'] = (filter_data['close'] - filter_data['close_t-1'])/filter_data['close_t-1']
        annual_std = filter_data[~filter_data['return'].isna()]['return'].std()*np.sqrt(250)
        sharpe = annual_return/annual_std
        return annual_return, sharpe

In [59]:
ret_3m = []
ret_6m = []
ret_1y = []
ret_2y = []
ret_3y = []
ret_5y = []
ret_10y = []
ret_15y = []
sharpes_5y = []
sharpes_10y = []
sharpes_15y = []
etfs = []
start_date = '2000-01-01'
end_date = '2020-02-12'
for each in etf_list['symbol'].values:
# for each in ['LUMN']:
    min_max = pd.read_sql_query(f"select min(date), max(date) from mkt where symbol = '{each}' and date <= '{end_date}' and date >= '{start_date}'", \
                                con = engine)
    min_date = min_max['min'].values[0]
    max_date = min_max['max'].values[0]
    
    if min_date is None or max_date is None:
        continue
    
    length = (dt.datetime.now().date() - min_date).days
    
    before_1y = max(min_date, max_date - dt.timedelta(365))
    before_2y = max(min_date, max_date - dt.timedelta(365*2))
    before_3y = max(min_date, max_date - dt.timedelta(365*3))
    before_5y = max(min_date, max_date - dt.timedelta(365*5))
    before_10y = max(min_date, max_date - dt.timedelta(365*10))
    before_15y = max(min_date, max_date - dt.timedelta(365*15))
    before_6m = max(min_date, max_date - dt.timedelta(365//2))
    before_3m = max(min_date, max_date - dt.timedelta(365//4))

    df = pd.read_sql_query(f"select * from mkt where symbol = '{each}' and date <= '{end_date}' and date >= '{start_date}'", con = engine)
    df = df.dropna()
    
    if len(df) == 0:
        continue
    
    etfs.append(each)
    
    annual_return, sharpe = update_return_sharpe(df, before_3m, min_date, end_date)
    ret_3m.append(annual_return)
    
    annual_return, sharpe = update_return_sharpe(df, before_6m, min_date, end_date)
    ret_6m.append(annual_return)
    
    annual_return, sharpe = update_return_sharpe(df, before_1y, min_date, end_date)
    ret_1y.append(annual_return)

    annual_return, sharpe = update_return_sharpe(df, before_2y, min_date, end_date)
    ret_2y.append(annual_return)
    
    annual_return, sharpe = update_return_sharpe(df, before_3y, min_date, end_date)
    ret_3y.append(annual_return)
    
    annual_return, sharpe = update_return_sharpe(df, before_5y, min_date, end_date)
    ret_5y.append(annual_return)
    sharpes_5y.append(sharpe)
    
    annual_return, sharpe = update_return_sharpe(df, before_10y, min_date, end_date)
    ret_10y.append(annual_return)
    sharpes_10y.append(sharpe)
    
    annual_return, sharpe = update_return_sharpe(df, before_15y, min_date, end_date)
    ret_15y.append(annual_return)
    sharpes_15y.append(sharpe)

<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std
<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std
<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std
<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std
<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std
<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std
<ipython-input-39-337e887e3fb0>:13: RuntimeWarning: invalid value encountered in double_scalars
  sharpe = annual_return/annual_std


In [60]:
result = pd.DataFrame({'symbol': etfs, 'ret_1y': ret_1y, 'ret_2y': ret_2y, \
                       'ret_3y': ret_3y, 'ret_5y': ret_5y, 'ret_10y': ret_10y, \
                       'ret_15y': ret_15y, 'ret_3m': ret_3m, 'ret_6m': ret_6m, \
                       'sharpe_5y': sharpes_5y, 'sharpe_10y': sharpes_10y, 'sharpe_15y': sharpes_15y})

In [25]:
win15y

,symbol,ret_1y,ret_2y,ret_3y,ret_5y,ret_10y,ret_15y,ret_3m,ret_6m,sharpe_5y,sharpe_10y,sharpe_15y
1284,AMZN,0.051059,0.336130,0.226280,0.338051,0.316084,0.374761,-0.067257,-0.009055,1.154881,1.009075,0.986178
2023,MA,0.248679,0.184864,0.240717,0.320764,0.273707,0.333647,0.070174,0.436341,1.089869,1.006340,0.970536
5747,AAPL,0.369224,0.666265,0.421961,0.402938,0.262150,0.311363,0.462301,0.180806,1.339732,0.916213,0.968972
5210,TYL,0.375466,0.408855,0.292899,0.244233,0.340789,0.278037,0.736957,0.329419,0.945091,1.208254,0.930812
497,IDXX,0.698705,0.584438,0.418345,0.482216,0.326263,0.254411,1.303065,0.889108,1.585761,1.177183,0.874636
1420,BKNG,0.307714,0.075817,0.024765,0.098678,0.152594,0.339607,-0.363009,0.195889,0.306931,0.483209,0.865346
3289,CPRT,0.570723,0.380472,0.363981,0.414312,0.298642,0.226350,0.933310,0.750410,1.446763,1.173659,0.835044
6509,MTD,0.570479,0.388143,0.355964,0.288771,0.257021,0.235323,0.568164,0.552134,1.072302,0.986510,0.834727
1262,WST,0.526165,0.724288,0.542993,0.381240,0.343137,0.227101,1.409741,0.911624,1.394416,1.357969,0.826693
2333,EW,0.427759,0.249193,0.328668,0.239509,0.248904,0.253164,0.970529,0.868852,0.757821,0.768545,0.826527


In [15]:
win5y = result[result['sharpe_5y'] == result['sharpe_5y']].sort_values(by = 'sharpe_5y', ascending = False).head(30)
win5y_set = set(win5y['symbol'])
win5y

,symbol,ret_1y,ret_2y,ret_3y,ret_5y,ret_10y,ret_15y,ret_3m,ret_6m,sharpe_5y,sharpe_10y,sharpe_15y
1969,HCIIW,10046.871433,10046.871433,10046.871433,10046.871433,0.935574,NaN,8930.335722,10046.871433,7632.015497,0.426754,NaN
1754,SHOP,0.460345,1.083815,1.212224,1.120510,NaN,NaN,2.164425,0.708922,2.179322,NaN,NaN
6661,JYNT,4.396942,1.042452,1.118844,1.019988,NaN,NaN,3.127805,3.999637,1.878151,NaN,NaN
3881,ENPH,2.112748,1.491519,2.211913,1.518510,NaN,NaN,2.693423,0.035011,1.788333,NaN,NaN
642,GNRC,1.640658,1.414607,0.965711,0.631000,0.364406,NaN,1.741416,1.607061,1.723336,0.924476,NaN
5625,APPS,3.481208,2.326632,2.485466,1.233612,0.329010,0.161178,-0.397729,0.107223,1.682262,0.457110,0.137871
2597,SQ,0.894488,0.739660,0.541968,0.883718,NaN,NaN,0.059583,0.239058,1.669818,NaN,NaN
1330,FIVN,0.659296,0.860451,0.821864,0.733957,NaN,NaN,0.520100,0.444054,1.623832,NaN,NaN
4116,ZYXI,-0.271691,0.226064,0.662919,1.370214,0.320094,0.264979,-0.186144,-0.413124,1.587966,0.292504,0.224229
497,IDXX,0.698705,0.584438,0.418345,0.482216,0.326263,0.254411,1.303065,0.889108,1.585761,1.177183,0.874636


In [16]:
win10y = result[result['sharpe_10y'] == result['sharpe_10y']].sort_values(by = 'sharpe_10y', ascending = False).head(30)
win10y_set = set(win10y['symbol'])
win10y

,symbol,ret_1y,ret_2y,ret_3y,ret_5y,ret_10y,ret_15y,ret_3m,ret_6m,sharpe_5y,sharpe_10y,sharpe_15y
1262,WST,0.526165,0.724288,0.542993,0.381240,0.343137,0.227101,1.409741,0.911624,1.394416,1.357969,0.826693
5697,ADBE,0.395336,0.439040,0.355626,0.443979,0.364008,0.217928,1.308666,0.741569,1.390908,1.238042,0.653426
5576,POOL,0.503830,0.583966,0.444890,0.357513,0.327786,0.178256,0.535694,0.755508,1.239208,1.221511,0.554611
5210,TYL,0.375466,0.408855,0.292899,0.244233,0.340789,0.278037,0.736957,0.329419,0.945091,1.208254,0.930812
6054,DPZ,0.355918,0.457658,0.246180,0.286927,0.344226,0.228552,1.144324,0.990702,0.937661,1.187157,0.592141
497,IDXX,0.698705,0.584438,0.418345,0.482216,0.326263,0.254411,1.303065,0.889108,1.585761,1.177183,0.874636
3289,CPRT,0.570723,0.380472,0.363981,0.414312,0.298642,0.226350,0.933310,0.750410,1.446763,1.173659,0.835044
2127,SNPS,0.441366,0.469638,0.471171,0.391501,0.282272,0.202289,0.905256,0.192208,1.393017,1.162906,0.782375
6052,ODFL,0.467701,0.551936,0.405424,0.418221,0.328717,0.246947,0.118338,0.818729,1.437355,1.155743,0.669413
546,RGEN,0.621836,0.597901,0.716614,0.519587,0.530326,0.345756,0.932678,0.413018,1.288985,1.148812,0.735226


In [61]:
win15y = result[result['sharpe_15y'] == result['sharpe_15y']].sort_values(by = 'sharpe_15y', ascending = False).head(30)
win15y_set = set(win15y['symbol'])
win15y

,symbol,ret_1y,ret_2y,ret_3y,ret_5y,ret_10y,ret_15y,ret_3m,ret_6m,sharpe_5y,sharpe_10y,sharpe_15y
4159,TYL,0.605197,0.324770,0.292821,0.236524,0.330054,0.291122,1.150733,0.806096,0.973144,1.236487,0.990220
4564,AAPL,0.899991,0.414150,0.346308,0.206771,0.272001,0.298880,1.359008,1.584673,0.839985,1.063428,0.933267
1145,BKNG,0.033919,0.046128,0.059543,0.121000,0.247805,0.345372,0.238468,0.073875,0.473331,0.816576,0.896850
3489,NFLX,0.054993,0.211887,0.381492,0.413581,0.446325,0.436293,2.277051,0.607970,1.013677,0.869126,0.843404
2635,CPRT,0.909455,0.541569,0.512954,0.396592,0.276052,0.209959,1.273814,0.892570,1.790189,1.244272,0.826724
1038,AMZN,0.314355,0.246077,0.369035,0.411106,0.335446,0.311227,1.320077,0.496313,1.439959,1.088523,0.815206
1854,ACGL,0.518162,0.271815,0.147214,0.185460,0.190419,0.166522,0.814223,0.501224,1.120588,1.156007,0.802832
396,IDXX,0.347343,0.269653,0.257818,0.287136,0.264075,0.218607,0.428319,0.101488,1.031954,1.041438,0.796405
4777,WCN,0.178255,0.213893,0.229481,0.265231,0.210674,0.166502,0.618615,0.229287,1.597102,1.222680,0.792643
1453,SBAC,0.589359,0.351108,0.384491,0.190681,0.237369,0.263935,1.586981,0.318207,0.843007,1.100747,0.787849


In [18]:
win10y_set.intersection(win5y_set).intersection(win15y_set)

{'CPRT', 'IDXX', 'WST'}

In [53]:
result[(result['ret_1y'] > result['ret_1y'].quantile(0.75)) & (result['ret_2y'] > result['ret_2y'].quantile(0.75)) \
       & (result['ret_3y'] > result['ret_3y'].quantile(0.75)) & (result['ret_4y'] > result['ret_4y'].quantile(0.75)) \
       & (result['ret_5y'] > result['ret_5y'].quantile(0.75)) & (result['ret_3m'] > result['ret_3m'].quantile(0.67)) \
       & (result['ret_6m'] > result['ret_6m'].quantile(0.67))]

,symbol,ret_1y,ret_2y,ret_3y,ret_4y,ret_5y,ret_3m,ret_6m,sharpe
389,QTEC,0.406504,0.360731,0.269677,0.281274,0.208975,0.502149,0.285163,0.916469
555,SPXL,1.298805,0.477808,0.321866,0.358385,0.342030,0.726417,1.422405,0.663742
628,GRID,0.655724,0.413166,0.245510,0.207152,0.124238,0.404862,0.340316,0.516226
705,RPG,0.399407,0.269896,0.184248,0.178187,0.155789,0.617642,0.357159,0.781792
758,PHO,0.453645,0.258251,0.217082,0.183189,0.121990,0.375309,0.455923,0.603437
786,UPRO,1.306874,0.482932,0.326111,0.372684,0.356083,0.728528,1.427896,0.689044
1073,SOXL,1.914556,1.016266,0.615702,0.768112,0.536126,1.397806,0.408785,0.671479
1129,EQWL,0.399669,0.197465,0.141172,0.144212,0.129463,0.168471,0.400818,0.781580
1267,IWR,0.402575,0.192353,0.139062,0.127796,0.120567,0.106477,0.315659,0.645212
1272,TQQQ,1.209762,1.014639,0.616320,0.671458,0.544594,1.450806,0.858799,0.930857
